## S5_LGB_main.py

In [1]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc,os,random
import time,datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from utils import *
from model import *
root = args.root
seed = args.seed

available gpus: [0]


In [2]:
df = pd.read_feather(f"{root}/all_feature.feather")

In [3]:
train_y = pd.read_csv(f"/mnt/sdb/KAGGLE_DATA/amex-default-prediction/train_labels.csv")

In [4]:
train = df[:train_y.shape[0]]
train["target"] = train_y["target"]
test = df[train_y.shape[0]:].reset_index(drop=True)
del df; gc.collect()

0

In [5]:
print(train.shape, test.shape)

(458913, 6390) (924621, 6389)


In [6]:
lgb_config = {
    "lgb_params": {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting": "dart",
        "max_depth": -1,
        "num_leaves": 64,
        "learning_rate": 0.035,
        "bagging_freq": 5,
        "bagging_fraction": 0.75,
        "feature_fraction": 0.05,
        "min_data_in_leaf": 256,
        "max_bin": 63,
        "min_data_in_bin": 256,
        #"min_sum_hessian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        "lambda_l1": 0.1,
        "lambda_l2": 30,
        "num_threads": 16,
        "verbosity": 1
    },
    "feature_name": [],
    "rounds": 4500,
    "early_stopping_rounds": 100,
    "verbose_eval": 50,
    "folds": 5,
    "seed": seed,
}

### trainとtest同時にやろうとするとメモリ終わるので、分けてやる必要アリ

In [7]:
del test; gc.collect()

0

In [8]:
lgb_config['feature_name'] = [col for col in train.columns if col not in [id_name,label_name,'S_2'] and 'target' not in col]
Lgb_train_and_predict(train=train, test=None, config=lgb_config, aug=None, run_id='LGB_with_manual_feature')

 Lgb_train_and_predict : LGB_with_manual_feature
     ... start train
     ... folds defined
        ... fold : 0


cp: cannot stat './*.sh': No such file or directory


[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.571873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 212196
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 6285
[50]	training's binary_logloss: 0.399496	valid_1's binary_logloss: 0.399914
[100]	training's binary_logloss: 0.339088	valid_1's binary_logloss: 0.339829
[150]	training's binary_logloss: 0.313336	valid_1's binary_logloss: 0.314433
[200]	training's binary_logloss: 0.300538	valid_1's binary_logloss: 0.30186
[250]	training's binary_logloss: 0.272821	valid_1's binary_logloss: 0.274745
[300]	training's binary_logloss: 0.258082	valid_1's binary_logloss: 0.260577
[350]	training's binary_logloss: 0.24827	valid_1's binary_logloss: 0.251258
[400]	training's binary_logloss

(                                              customer_ID    target
 0       0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.000397
 1       00000fd6641609c6ece5454664794f0340ad84dddce9a2...  0.001275
 2       00001b22f846c82c51f6e3958ccd81970162bae8b007e8...  0.000982
 3       000041bdba6ecadd89a52d11886e8eaaec9325906c9723...  0.002219
 4       00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...  0.000888
 ...                                                   ...       ...
 458908  ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...  0.002091
 458909  ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...  0.028120
 458910  ffff9984b999fccb2b6127635ed0736dda94e544e67e02...  0.001113
 458911  ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...  0.121225
 458912  fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...  0.002370
 
 [458913 rows x 2 columns],
 None,
 (0.7982306864785512, 0.7983499947470201))

In [9]:
## lgb_config['feature_name'] = [col for col in train.columns if col not in [id_name,label_name,'S_2']]
## Lgb_train_and_predict(train,test,lgb_config,aug=None,run_id='LGB_with_manual_feature_and_series_oof')